In [14]:
import pandas as pd
import numpy as np
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
import random

import glob
from tqdm import tqdm
from pathlib import Path
import json
import joblib

from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor, as_completed
import os

import subprocess

from collections import defaultdict
from typing import Optional

from scipy import sparse
from scipy.sparse import coo_matrix, save_npz, load_npz
import implicit

from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, StandardScaler

from catboost import CatBoostClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer, f1_score
from sklearn.linear_model import LogisticRegression

**Вспомогательные функции**

In [15]:
# read table from SSD/HDD
def read_df_from_parquets(path_to_parquets: str, n=None):
    files = glob.glob(path_to_parquets + '/*.parquet')
    count_to_read = len(files) if n is None else min(n, len(files))
    return pl.read_parquet(files[:count_to_read])

# scan table from SSD/HDD
def scan_df_from_parquets(path_to_parquets: str, n=None):
    files = glob.glob(path_to_parquets + '/*.parquet')
    count_to_scan = len(files) if n is None else min(n, len(files))
    return pl.scan_parquet(files[:count_to_scan])

# write table to SSD/HDD with chunks
def write_parquet_in_chunks(df: pl.DataFrame, path: str, chunk_size=100000):
    out = Path(path)
    out.mkdir(parents=True, exist_ok=True)
    for i in range(0, df.height, chunk_size):
        chunk = df.slice(i, chunk_size)
        file_path = out / f"part_{i // chunk_size:04d}.parquet"
        chunk.write_parquet(file_path)

### 1. Обработка таблицы логов

Сначала внутри каждого .parquet файла происходит агрегация и создается сгрупированный файл .parquet в папку agg_interactions

In [3]:
def analyze_logs_table_one_by_one(
    path_to_orders: str,
    path_to_tracker: str,
    output_path='./data/agg_interactions',
):
    agg_interactions_dir = Path(output_path)
    agg_interactions_dir.mkdir(exist_ok=True)

    df_orders = (
         scan_df_from_parquets(path_to_orders)
         .select(["user_id", "item_id", "last_status"])
         .with_columns([
               pl.col("user_id").cast(pl.Int64),
                pl.col("item_id").cast(pl.Int64),
                pl.col("last_status")
         ])
         .filter(pl.col("last_status") != "proccesed_orders")
    )
    
    def encode_actions(df: pl.LazyFrame):
        return df.with_columns([
            (pl.col("action_type") == "view_description").cast(pl.Int32).alias("action_type_view_description"),
            (pl.col("action_type") == "to_cart").cast(pl.Int32).alias("action_type_to_cart"),
            (pl.col("action_type") == "page_view").cast(pl.Int32).alias("action_type_page_view"),
            (pl.col("action_type") == "favorite").cast(pl.Int32).alias("action_type_favorite"),
            (pl.col("action_type") == "unfavorite").cast(pl.Int32).alias("action_type_unfavorite"),
            (pl.col("action_type") == "review_view").cast(pl.Int32).alias("action_type_review_view"),
            (pl.col("action_type") == "remove").cast(pl.Int32).alias("action_type_remove"),
            pl.when(pl.col("last_status") == "canceled_orders").then(0)
              .when(pl.col("last_status") == "delivered_orders").then(1)
              .otherwise(0)
              .cast(pl.Int8)
              .alias("last_status")
        ]).drop("action_type")

    tracker_files = sorted(glob.glob(path_to_tracker))
    chunk_size = 1
    for i in range(0, len(tracker_files), chunk_size):
        files_chunk = tracker_files[i:i+chunk_size]
        print(f"Processing chunk {i//chunk_size + 1}/{(len(tracker_files)+chunk_size-1)//chunk_size}")
    
        chunk_df = (pl.scan_parquet(files_chunk)
                    .with_columns([
                        pl.col("user_id").cast(pl.Int64),
                        pl.col("item_id").cast(pl.Int64),
                    ]))
    
        temp = (
            chunk_df
            .join(df_orders, on=["item_id","user_id"], how="left")
        )
        temp = encode_actions(temp)
    
        temp_agg = temp.group_by(["user_id","item_id"]).agg([
            pl.col([
                "action_type_view_description",
                "action_type_to_cart",
                "action_type_page_view",
                "action_type_favorite",
                "action_type_unfavorite",
                "action_type_review_view",
                "action_type_remove"
            ]).sum(),
            pl.col("last_status").first()
        ])
    
        output_file = agg_interactions_dir / f"agg_chunk_{i//chunk_size}.parquet"
        temp_agg.collect(engine='streaming').write_parquet(output_file)
        print(f"Saved {output_file}")

In [4]:
def compress_logs_table(
    agg_interactions_path='./data/agg_interactions_dir',
    interactions_path='./data/interactions'
):
    chunks_dir = Path(agg_interactions_path)

    all_files = sorted(glob.glob(str(chunks_dir / "*.parquet")))
    
    out_path = Path(interactions_path)
    out_path.mkdir(exist_ok=True)
    chunk_size = 2
    
    for i in tqdm(range(0, len(all_files), chunk_size)):
        batch = all_files[i:i+chunk_size]
        
        # читаем сразу все parquet из батча
        df = pl.scan_parquet(batch)
        agg_data = (
            df
            .group_by(["user_id","item_id"])
            .agg([
                pl.col([
                    "action_type_view_description",
                    "action_type_to_cart",
                    "action_type_page_view",
                    "action_type_favorite",
                    "action_type_unfavorite",
                    "action_type_review_view",
                    "action_type_remove"
                ]).sum(),
                pl.col("last_status").first()
            ])
        )
        del df
        out_file = out_path / f"grouped_batch_{i//chunk_size + 1}.parquet"
        agg_data.sink_parquet(out_file)
        del agg_data

In [ ]:
def analyze_logs_table(
    path_to_orders='./data/ml_ozon_recsys_train_final_apparel_orders_data',
    path_to_tracker='./data/ml_ozon_recsys_train_final_apparel_tracker_data',
    agg_interactions_path='./data/agg_interactions',
    interactions_path='./data/interactions'
):
    analyze_logs_table_one_by_one(path_to_orders, path_to_tracker, agg_interactions_path)
    compress_logs_table(agg_interactions_path, interactions_path)

analyze_logs_table()

Processing chunk 1/1


#### 2. Нахождение коэффициентов для рейтинга с помощью логистической регрессии

In [6]:
def find_weights_for_rating(
    feature_cols: list,
    interactions_path='./data/interactions',
):
    interactions = read_df_from_parquets(interactions_path)
    
    X_train = interactions.select(feature_cols).to_numpy()
    y_train = interactions['last_status'].to_numpy()

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_train)

    model = LogisticRegression()
    model = model.fit(X_scaled, y_train)

    coefs = model.coef_.ravel()

    weights = dict(zip(feature_cols, coefs))
    return weights

feature_cols = [
    "action_type_view_description",
    "action_type_to_cart",
    "action_type_page_view",
    "action_type_favorite",
    "action_type_unfavorite",
    "action_type_review_view",
    "action_type_remove"
]

weights = find_weights_for_rating(feature_cols)
print(weights)

{'action_type_view_description': -0.09608848421927428, 'action_type_to_cart': 2.4621577568713238, 'action_type_page_view': 0.1828402680655119, 'action_type_favorite': -0.22149147361203042, 'action_type_unfavorite': 0.18611649944352027, 'action_type_review_view': 0.026484669994032747, 'action_type_remove': -1.9714975325140849}


#### 3. Генерируем матрицу R для ALS

На пересечении R[i, j] лежит линейная комбинация действий пользователя i с товаром j с коэффициентами, полученными в прошлом пункте

In [11]:
def gen_matrix_R(
    train_data_path,
    output_path,
    weights,
    n_parquets=10
):
    all_files = sorted(glob.glob(str(train_data_path /"*.parquet")))
    print(all_files)
    
    out_dir = Path(output_path)
    out_dir.mkdir(exist_ok=True)

    r = sum(pl.col(col)*coef for col, coef in weights)
    iter = 0
    for file in tqdm(all_files):
        iter += 1
        data = pl.scan_parquet(file).with_columns([
            r.cast(pl.Float32).alias("rating")
        ]).drop(columns).drop("last_status")
    
        data.sink_parquet(out_dir / f"{iter}.parquet")

In [ ]:
gen_matrix_R(Path("./data/interactions"), Path("./data/matrix_R_coord"), weights)

#### 4. Обучаем ALS

In [16]:
def learn_ALS_with_R_matrix(
    path_to_R='./data/full_matrix2.npz',
    *,
    factors=200,
    iterations=50,
    regularization=0.05
):
    R = sparse.load_npz(path_to_R)
    model = implicit.als.AlternatingLeastSquares(factors, iterations,regularization)
    model.fit(R)
    model.save(f'./data/als_model_{factors}_{iterations}')

In [ ]:
learn_ALS_with_R_matrix()

#### 5. Отбор кандидатов (товаров) для каждого пользователя

In [7]:
class CategoryTree:
    def __init__(self):
        self.parent = defaultdict(int)
        self.children = defaultdict(set)
        self.subtree = defaultdict(set)
        self.subtree_children = defaultdict(set)

    @classmethod
    def from_df(cls, category_tree_df: pl.DataFrame):
        self = cls()
        for path in category_tree_df['ids']:
            for i in range(len(path) - 1):
                cur_id = int(path[i])
                parent_id = path[i + 1]

                if cur_id not in self.parent:
                    self.parent[cur_id] = parent_id

                if parent_id != -1:
                    self.children[parent_id].add(cur_id)

                self.children.setdefault(cur_id, set())

        for p in {v for v in self.parent.values() if v != -1}:
            self.children.setdefault(p, set())
            self.parent.setdefault(p, -1)

        return self

    def is_leaf(self, catalog_id: int):
        if catalog_id not in self.children:
            raise RuntimeError("There is no catalog_id in category tree")
        return len(self.children[catalog_id]) == 0

    def get_subtree(self, catalog_id: int):
        # Use pre-calculated subtree
        if catalog_id in self.subtree:
            return self.subtree[catalog_id]

        subtree = set()
        subtree.add(catalog_id)

        for son in self.children[catalog_id]:
            subtree |= self.get_subtree(son)

        self.subtree[catalog_id] = subtree
        return subtree

    def get_subtree_children(self, catalog_id: int):
        # Use pre-calculated subtree
        if catalog_id in self.subtree_children:
            return self.subtree_children[catalog_id]

        subtree_children = set()
        if self.is_leaf(catalog_id):
            subtree_children.add(catalog_id)

        for son in self.children[catalog_id]:
            subtree_children |= self.get_subtree_children(son)

        self.subtree_children[catalog_id] = subtree_children
        return subtree_children

    # Leaves only leaves
    def find_relevant_categories(self, users_interesting_categories: list, k_nearest=5, n=30):
        categories = set(users_interesting_categories)
        relevant_categories = set()

        for catalog_id in categories:
            if catalog_id is None:
                continue
            
            relevant_categories |= self.get_subtree_children(catalog_id)
            # if category is a leaf then we can choose k nearest leaves
            if self.is_leaf(catalog_id):
                parent_id = self.parent.get(catalog_id, -1)
                if parent_id != -1:
                    # choose k random children of the subtree of the parent without current 'catalog_id'
                    siblings = list(self.get_subtree_children(parent_id) - {catalog_id})
                    relevant_categories.update(random.sample(siblings, min(k_nearest, len(siblings))))

        # Take N random relevant categories
        relevant_categories = list(relevant_categories)
        return random.sample(relevant_categories, min(n, len(relevant_categories)))

    def get_users_relevant_categories(self, users_interacted_categories: pl.DataFrame,
                                      interacted_categories_column='interacted_categories',
                                      n=30):
        return (
            users_interacted_categories
            .with_columns(
                pl.col(interacted_categories_column)
                .map_elements(lambda categories: self.find_relevant_categories(categories),
                              return_dtype=pl.List(pl.Int64))
                .alias('relevant_categories')
            )
        )

Создаём дерево категорий и инициализируем его из приложенного датасета

In [8]:
category_tree = CategoryTree.from_df(read_df_from_parquets('./data/ml_ozon_recsys_train_final_categories_tree'))

Делаем сводную таблицу с релевантными категориями и брендами, с которыми взаимодействовал тот или иной пользователь

In [9]:
def get_users_relevant_categories_and_brands(
    path_to_interactions: str,
    path_to_items: str,
    weights: dict,
    category_tree: CategoryTree,
    *,
    percent_top_items=0.40,
    n_top_categories=50,
    n_interactions_files=None,
    n_items_files=None
):
    interactions = scan_df_from_parquets(path_to_interactions, n_interactions_files)
    items = scan_df_from_parquets(path_to_items, n_items_files)

    items_with_brands = items.with_columns(
        pl.col("attributes").map_elements(
            lambda lst: next(
                (x["attribute_value"] for x in lst if x["attribute_name"] == "Brand"),
                None
            ),
            return_dtype=pl.Utf8,
        ).alias("brand")
    )
    
    interactions_rated_with_brand_and_category = (
        interactions
        .with_columns(
            pl.sum_horizontal([
                pl.col(c).cast(pl.Float64).fill_null(0.0) * float(w)
                for c, w in weights.items()
            ]).alias("rating")
        )
        .join(
            items_with_brands
            .select([
                "item_id",
                pl.col("catalogid").alias("category"),
                "brand"
            ]),
            on="item_id",
            how="left"
        )
        .select("user_id", "item_id", "category", "brand", "rating")
    )

    # Для каждого пользователя находим самые интересные ему товары (берем долю S наилучших)
    top_items_for_users = (
        interactions_rated_with_brand_and_category
        .sort(["user_id", "rating"], descending=[False, True])
        .with_columns([
            pl.len().over("user_id").alias("n"),
            pl.arange(0, pl.len()).over("user_id").alias("rank"),
        ])
        .with_columns(
            pl.max_horizontal(pl.lit(1), (pl.col("n") * percent_top_items).ceil().cast(pl.Int64)).alias("k")
        )
        .filter(pl.col("rank") < pl.col("k"))
        .drop(['rank', 'n', 'k'])
    )

    # Для каждого пользователя берём N наиболее интересных категорий 
    # Рейтинг категории для пользователя = суммарный рейтинг всех товаров данной категории,
    # с которыми взаимодействовал пользователь
    top_n_categories = (
        top_items_for_users
        .select(['user_id', 'category', 'rating'])
        .group_by(['user_id', 'category'])
        .agg(pl.col('rating').sum().alias("summary_rating"))
        .group_by('user_id')
        .agg(
            pl.col('category')
                .sort_by(pl.col('summary_rating'), descending=True)
                .head(n_top_categories)
                .alias('interacted_categories')
        )
    )

    # Находим ближайших соседей в дереве
    relevant_top_n_categories = category_tree.get_users_relevant_categories(top_n_categories)

    # Аналогично находим наиболее интересные бренды для пользователя
    # Бренды срезать не будем, просто возьмём наиболее релевантные товары
    top_brands = (
        top_items_for_users
        .filter(pl.col("brand").is_not_null() & (pl.col("brand") != "Нет бренда"))
        .group_by("user_id")
        .agg(pl.col("brand").unique().alias("relevant_brands"))
    )

    # Собираем в одну таблицу
    users_relevant_categories_and_brands = (
        relevant_top_n_categories
        .join(top_brands, on='user_id', how='left')
        .select(['user_id', 'relevant_categories', 'relevant_brands'])
    )

    return users_relevant_categories_and_brands

In [ ]:
users_relevant_categories_and_brands = get_users_relevant_categories_and_brands(
    './data/interactions',
    './data/ml_ozon_recsys_train_final_apparel_items_data',
    weights,
    category_tree
).collect(engine='streaming')

print(users_relevant_categories_and_brands.head())

Сохраняем на диск

In [ ]:
write_parquet_in_chunks(users_relevant_categories_and_brands, './data/users_relevant_categories_and_brands')

#### 6. Делаем предсказания

In [ ]:
unknown = joblib.load("unknown_ids.joblib")
encoded = joblib.load("encoded_ids.joblib")
R = sparse.load_npz("./data/full_matrix2.npz")
le_i = joblib.load("./models/label_enc_items.joblib")
model = implicit.als.AlternatingLeastSquares()
model = model.load("./models/als_model_50_20_test.npz")
categories_enc = pl.scan_parquet("./data/users_relevant_categories_encoded/*.parquet").collect()
catalog2items = pl.scan_parquet("./data/catalog2items.parquet").collect()
catalog2items = (catalog2items.with_columns(pl.Series("item_id", 
                                                      [le_i.transform(catalog2items["item_id"][i]) for i in range(len(catalog2items))]))
                                                                                            .to_pandas().set_index("catalogid"))
                                                                             .to_pandas().set_index("catalogid"))
catalog2items_dict = catalog2items.to_dict()["item_id"]
user2catalogs = (categories_enc.group_by("user_id")
                             .agg(pl.col("relevant_categories").unique()))
user2catalogs = user2catalogs.to_pandas().set_index("user_id").map(lambda x: x[0])
user2catalogs_dict = user2catalogs.to_dict()["relevant_categories"]

In [ ]:
from sklearn.preprocessing import LabelEncoder


def get_items_for_catalog(catalogid):
    # res = catalog2items_dict.get(catalogid)
    # return res[:min(len(res), 10_000)]
    return catalog2items_dict.get(catalogid)


def get_item_candidates_for_user(user_id, R_row):
    catalogs =  user2catalogs_dict[user_id]
    catalogs = catalogs[:min(len(catalogs),30)]
    
    candidates = np.concatenate([catalog2items_dict.get(c, np.empty(0, dtype=np.int32)) for c in catalogs])
    candidates = np.unique(candidates) 


    return np.setdiff1d(candidates, R_row.indices, assume_unique=True)


def make_prediction(user_emb:np.array, item_candidates, item_candidates_emb:np.array):
    scores = item_candidates_emb @ user_emb  
    top_idx = np.argpartition(scores, -100)[-100:]
    top_idx = top_idx[scores[top_idx].argsort()[::-1]]

    return item_candidates[top_idx]


def predict_batch(R_trunc, batch_user_ids):
    results = {}
    item_factors = model.item_factors
    user_factors = model.user_factors
    for i in range(len(batch_user_ids)):
        user = batch_user_ids[i]
        candidates = get_item_candidates_for_user(user, R_trunc[i])

        item_candidates_emb = item_factors[candidates]
        res = make_prediction(user_factors[user], candidates, item_candidates_emb)
        results[user] = res
    return results

In [ ]:
# os.environ["OMP_NUM_THREADS"] = "1"
# os.environ["MKL_NUM_THREADS"] = "1"

def chunks(lst, batch_size):
    for i in range(0, len(lst), batch_size):
        yield lst[i:i + batch_size]


batch_size = 300    
all_results = {}
print("total iters = ", 470000 / batch_size)

# for batch_users in tqdm(chunks(encoded, batch_size)):
    # all_results.update(predict_batch2( R[batch_users], batch_users,  le_i, None))
iterations = 0
with ThreadPoolExecutor(max_workers=10) as executor:
    futures = []
    for batch_users in tqdm(chunks(encoded, batch_size)):
        futures.append(
            executor.submit(
                predict_batch, R[batch_users], batch_users
            )
        )

    for future in tqdm(as_completed(futures)):
        iterations += 1
        all_results.update(future.result())
        # if iterations % 100 == 0:
        #     df = pd.DataFrame({
        #         'values': list(all_results.values())
        #     }, index=list(all_results.keys()))
        #     df.to_csv(f'output_2_{iterations}.csv')

# with ProcessPoolExecutor(max_workers=6) as executor:
#     futures = {executor.submit(predict_batch, R[batch], batch, le_i, None): batch
#                for batch in tqdm(chunks(encoded, batch_size))}

#     for i, future in tqdm(enumerate(as_completed(futures), 1)):
#         all_results.update(future.result())
#         if i % 300 == 0:
#             pd.DataFrame.from_dict(all_results, orient="index").to_csv(f"output2_{i}.csv")


df = pd.DataFrame({
    'values': list(all_results.values())
}, index=list(all_results.keys()))
df.to_csv('output3.csv')
all_results

In [ ]:
user_decoder = joblib.load("models/ordinal_enc_users.joblib")
item_decoder = joblib.load("models/label_enc_items.joblib")

def decode_items(encoded_list):
        return item_decoder.inverse_transform(encoded_list)

def decode_and_merge(pred_df: pd.DataFrame, input_file: str) -> dict:
    # user_ids = user_decoder.categories_[0]
    decoded_users = user_decoder.inverse_transform(pred_df.index.to_numpy().reshape(-1,1)).ravel()
    # print(decoded_users)
    values = np.concatenate(pred_df["values"].to_numpy())
    # display(values)
    decoded = item_decoder.inverse_transform(values).reshape(-1,100)
    # print(decoded.shape)
    
    df_decoded = pd.DataFrame({"user_id": decoded_users,
                            "values": [row for row in decoded],
                               })
    
    extra_df = pd.read_csv(input_file, header=None)
    extra_df[1] =  extra_df[1].map(lambda x: np.array(list(map(int, x[1:-1].split()))))
    extra_df.columns = ["user_id", "values"]
    # merged_df = extra_df.merge(df_decoded, on="user_id", how="inner")
    merged_df = pd.concat([df_decoded, extra_df])
    # display(merged_df)

    
    result_dict = dict(zip(merged_df["user_id"], merged_df["values"]))
    
    return result_dict

decoded = decode_and_merge(df, 'unknown_users.csv')

def create_submission(recommendations, filename='submission5.csv'):

    print(f"Создаем файл submission: {filename}")
    
    submission_data = []
    for user_id, items in tqdm(recommendations.items(), desc="Формирование submission"):
        submission_data.append({
            'user_id': user_id,
            'item_id_1 item_id_2 ... item_id_100': ' '.join(map(str, items))
        })
    
    submission_df = pd.DataFrame(submission_data)
    submission_df.to_csv(filename, index=False)
    
    print(f"Сохранен submission с {len(submission_df):,} пользователями")
    print(f"Размер файла: {Path(filename).stat().st_size / 1024 / 1024:.1f} MB")
    
    return filename

submission_file = create_submission(decoded)